#Реализация программы по обработке данных банковских операций

# Загрузка данных

In [ ]:
# импортируем необходимые библиотеки
import pandas as pd
import json
import re

In [ ]:
# открытие файла и преобразование его в сплошную строку, т.к. функция json.loads 
# не может считать иной формат 
with open('transactions_final.json', "r") as f:
  src = f.read()
src = src.replace('\n', '')
src = src.replace('\t', '')

#загрузка строки, как json файла и снятие вложенности 'transactions'
trans = json.loads(src)
trans = trans['transactions']

#Запись в файл без обертки 'transactions', \n и \t
with open('trans.json', "w+") as f:
  f.write(json.dumps(trans))

In [ ]:
# загрузка файла и изменение индексов (pandas автоматически преобразовывает в даты)
df = pd.read_json('trans.json')
df.columns = list(map(int, trans.keys()))

In [ ]:
# Транспонирование матрицы для более удобного представление
df_T = df.T

# Проверка датасета на ошибки

In [ ]:
# Задание шаблонов

time = r"\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}"
card = r"\d{20}"
account_pass = r"\d{10}"
date = r"\d{4}-\d{2}-\d{2}"
client = r"\d-\d{5}"
word = r"[\w\s-]+"
phone = r"\+7\d{10}"
amount = r"\d+.\d{2}"
terminal = r"[A-Z]{3}\d{5}"
term_type = r"[A-Z]{3}"
address = r".+"

regular = [time, card, account_pass, date, client, word, word, word, date, 
           account_pass, date, phone, word, word, terminal, term_type, address, address]

In [ ]:
# Проверка на ошибки и удаление ненужных
invalid_trans = []

for trans in df_T.itertuples():
  trans_list =[trans.date, trans.card, trans.account, trans.account_valid_to, 
               trans.client, trans.last_name, trans.first_name, 
               trans.patronymic, trans.date_of_birth, trans.passport, 
               trans.passport_valid_to, trans.phone, trans.oper_type, 
               trans.oper_result, trans.terminal, trans.terminal_type, 
               trans.city, trans.address]
  if None in list(map(re.fullmatch, regular, trans_list)):
    print(trans) # вывод удаляемых транзакций
    invalid_trans += [trans.Index]

df_T.drop(invalid_trans, inplace=True)

Pandas(Index=3649243565, date='2020-05-11T08:38:27', card='52307315241694886580', account='7456779365', account_valid_to='2023-07-11', client='7-10589', last_name='Шестов', first_name='', patronymic='Афиногенов', date_of_birth='1971-04-30', passport='6235035467', passport_valid_to='2013-08-10', phone='+70551948129', oper_type='Пополнение', amount=78161.51, oper_result='Успешно', terminal='POS28311', terminal_type='POS', city='Шали', address='Шали, ул. Мельникова, д. 2')
Pandas(Index=3649245420, date='2020-05-13T03:03:36', card='59277321172104488678', account='7456781220', account_valid_to='2034-06-30', client='3-02535', last_name='Четвериков', first_name='', patronymic='Филяев', date_of_birth='1962-01-02', passport='5286715974', passport_valid_to='2028-03-23', phone='+70283070319', oper_type='Пополнение', amount=42686.41, oper_result='Отказ', terminal='POS43792', terminal_type='POS', city='Сухиничи', address='Сухиничи, ул. Стельмаха Михаила, д. 24')
Pandas(Index=3649245421, date='2020-

# Проверка данных

In [ ]:
lst_clients = list(set(list(df_T['client'])))

In [ ]:
# Проверка таблицы
df_T

,date,card,account,account_valid_to,client,last_name,first_name,patronymic,date_of_birth,passport,passport_valid_to,phone,oper_type,amount,oper_result,terminal,terminal_type,city,address
3649235840,2020-05-04T00:00:04,50785425162189603667,7456771640,2026-03-06,6-33721,Кадимов,Корнелиус,Ковалько,1993-09-17,2300733597,2027-12-11,+74304126656,Оплата,6734.1,Успешно,POS43792,POS,Нижний Новгород,"Нижний Новгород, ул. Короленковская, д. 5"
3649235841,2020-05-04T00:01:19,57642678631657633216,7456771641,2025-08-15,6-33721,Кадимов,Корнелиус,Ковалько,1993-09-17,2300733597,2021-10-19,+74304126656,Снятие,69108.86,Отказ,POS43792,POS,Заречный,"Заречный, ул. Селянская, д. 57"
3649235842,2020-05-04T00:02:51,50606101490380719590,7456771642,2034-09-08,2-10067,Котков,Валерьян,Байбаков,1946-09-08,3153290165,2006-07-07,+74832003245,Снятие,52699.45,Успешно,POS43792,POS,Донецк,"Донецк, ул. Возрождения, д. 23"
3649235843,2020-05-04T00:04:16,40801177229019601602,7456771643,2033-12-14,7-17459,Сафьянов,Кондрат,Романцов,1965-11-27,3307023178,2031-11-17,+75914405368,Оплата,9705.71,Успешно,POS28311,POS,Красноуфимск,"Красноуфимск, ул. Старошишковская, д. 77"
3649235844,2020-05-04T00:05:56,57447203234952512132,7456771644,2025-06-20,2-68359,Карельский,Алишер,Шавкунин,1984-09-12,6750185314,2030-01-09,+74862414292,Пополнение,82576.5,Отказ,POS75616,POS,Черкесск,"Черкесск, ул. Гинтовта, д. 56"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3649246068,2020-05-13T17:56:34,51455507818735472585,7456781868,2032-08-15,1-52990,Ходкевич,Джихангир,Сережников,1986-07-12,5338669072,2021-10-17,+70425393833,Снятие,45461.23,Отказ,POS43792,POS,Камбарка,"Камбарка, ул. Тикоцкого, д. 49"
3649246069,2020-05-13T17:57:42,56249165876813962365,7456781869,2028-01-05,9-44341,Глеб,Арам,Понькин,1941-02-06,8123902581,2028-02-08,+73271258441,Снятие,35450.79,Успешно,POS43792,POS,Мариинский Посад,"Мариинский Посад, ул. Юнкерова Николая, д. 96"
3649246070,2020-05-13T17:59:25,40085487866355277437,7456781870,2033-03-18,2-78222,Иноземцев,Филарет,Татьянищев,1932-04-12,3563544247,2014-04-02,+77123421714,Пополнение,84430.08,Успешно,POS43792,POS,Облучье,"Облучье, ул. Ирченко, д. 49"
3649246071,2020-05-13T18:00:38,56747369443847407380,7456781871,2032-02-12,3-90021,Стручков,Лукас,Арсенович,1968-06-01,2673115069,2023-11-21,+75194136280,Оплата,88799.29,Отказ,POS28311,POS,Хадыженск,"Хадыженск, ул. Кленовая, д. 88"


***Пример транзакции:***

{'date': '2020-05-01T00:00:29',

 'card': '59649132026167121328',
 
 'account': '40817810000001139973',
 
 'account_valid_to': '2036-01-16T00:00:00',
 
 'client': '3-95179',
 
 'last_name': 'Мисик',
 
 'first_name': 'Сергей',
 
 'patronymic': 'Николаевич',
 
 'date_of_birth': '1938-06-25T00:00:00',
 
 'passport': 7076445954,
 
 'passport_valid_to': '2022-11-09T00:00:00',
 
 'phone': '+79497481039',
 
 
 'oper_type': 'Пополнение',
 
 'amount': 31576.6,
 
 'oper_result': 'Отказ',
 
 'terminal': 'POS43792',
 
 'terminal_type': 'POS',
 
 'city': 'Славянск-на-Кубани',
 
 'address': 'Славянск-на-Кубани, ул. Клецкая, д. 86'}

# Реализация функций фильтрации

In [ ]:
def diff_city(df): #паттерн 1: посещение городов за малый промежуток времени
  lst_trans = list(df.index)
  lst_minutes = []
  lst_date = list(df['date'])
  lst_city = list(df['city'])
  res = []
  size_date = len(lst_date)
  for i in range(size_date):
    lst_minutes.append(int(lst_date[i].split('T')[1].split(':')[1]) + 60 * int(lst_date[i].split('T')[1][:2]) + 24 * 60 * (int(lst_date[i].split('T')[0][-1]) - 1)) #выделение минут
  for i in range(1, size_date):
    if (lst_minutes[i] - lst_minutes[i-1]) < 30 and lst_city[i] != lst_city[i-1]:  # проверка на условие (за 30 минут приехал в другой город)
      res += [lst_trans[i]] # номер транзакции
  return list(set(res))

def not_valid_account(df): #паттерн 2: не действующий аккаунт
  lst_trans = list(df.index)
  lst_month = []
  lst_date = list(df['date'])
  lst_valid = list(df['account_valid_to'])
  res = []
  size_date = len(lst_date)
  for i in range(size_date):
    lst_month.append(int(lst_date[i].split('-')[0]) * 12 + int(lst_date[i].split('-')[1])) # список с месяцами операций
    lst_valid[i] = int(int(lst_valid[i].split('-')[0]) * 12 + int(lst_valid[i].split('-')[1])) # срок валидности аккаунта
    if lst_valid[i] - lst_month[i] < -6: # проверка на просрочку в пол года (первые пол года человек может воспользоваться аккаунтом)
      res += [lst_trans[i]] # номер транзакции
  return list(set(res))

def not_valid_passport(df): # паттерн 3: недействительный паспорт
  lst_trans = list(df.index)
  lst_month = []
  lst_date = list(df['date'])
  lst_valid = list(df['passport_valid_to'])
  res = []
  size_date = len(lst_date)
  for i in range(size_date):
    lst_month.append(int(lst_date[i].split('-')[0]) * 12 + int(lst_date[i].split('-')[1])) # список с месяцами операций
    lst_valid[i] = int(int(lst_valid[i].split('-')[0]) * 12 + int(lst_valid[i].split('-')[1])) # срок валидности паспорта
    if lst_valid[i] - lst_month[i] < -3: # проверка на просрочку (первые 3 месяца человек может пользоваться картой)
      res += [lst_trans[i]] # номер транзакции
  return list(set(res))

def fast_trans(df): # паттерн 4: совершение транзакций за малый промежуток времени
  lst_trans = list(df.index)
  lst_minute = []
  lst_date = list(df['date'])
  res = []
  size_date = len(lst_date)
  for i in range(size_date):
    lst_minute.append(int(lst_date[i].split('T')[1].split(':')[-1]) + 60 * int(lst_date[i].split('T')[1].split(':')[1]) + 3600 * int(lst_date[i].split('T')[1][:2]) + 24 * 3600 * (int(lst_date[i].split('T')[0][-1]) - 1)) # время операции в минутах
  for i in range(1, size_date):
    if (lst_minute[i] - lst_minute[i-1]) < 90: # проверка на условие
      res += [lst_trans[i]] + [lst_trans[i-1]] # номер транзакции
  return list(set(res))

def period(df): # паттерн 5: проверка транзакций на периодичность
  lst_trans = list(df.index)
  lst_sec = []
  lst_diff = []
  lst_date = list(df['date'])
  res = []
  size_date = len(lst_date)
  size_trans = len(lst_trans)
  for i in range(size_date): #  ниже время операции в секундах
    lst_sec.append(int(lst_date[i].split('T')[1].split(':')[-1]) + 60 *int(lst_date[i].split('T')[1].split(':')[1]) + 60 * 60 * int(lst_date[i].split('T')[1][:2]) + 24 * 60 * 60 * (int(lst_date[i].split('T')[0][-1]) - 1))
  for i in range(1, size_trans):
    lst_diff.append(lst_sec[i] - lst_sec[i-1]) #вычисление разности времени транзакций
  size_diff = len(lst_diff)
  for i in range(1, size_diff):
    if lst_diff[i] - lst_diff[i-1] < 0.2 * lst_diff[i-1]: # проверка условия (с учетом погрешности)
      res += [lst_trans[i]] + [lst_trans[i-1]] # номер транзакции
  return list(set(res))

def average(df): # паттерн 6: проверка на сумму операции
  lst_trans = list(df.index)
  lst_money = list(df['amount'])
  res = []
  n = len(lst_money)
  if n > 2: # для 2х и меньше операций нет смысла вычислять эти данные (слишком маленькая выборка) 
    aver = sum(lst_money)/len(lst_money) # среднее значение суммы операции по клиенту
    deviation = 0
    for x_i in lst_money:
      deviation += (x_i - aver)**2
    deviation = (deviation / n) ** 0.5 # вычисляем несммещенную оценку дисперсии
    for i in range(n):
      if abs(lst_money[i] - aver) > deviation: # проверка условия
        res += [lst_trans[i]] # номер транзакции
  return list(set(res))

def time_to_sleep(df): # паттерн 7: проверка на совершение операции в необычное время с учетом возраста
  lst_trans = list(df.index)
  lst_age = list(df['date_of_birth'])
  lst_time = list(df['date'])
  res = []
  n = len(lst_age)
  for i in range(n):
    lst_age[i] = int(lst_time[i].split('-')[0]) - int(lst_age[i].split('-')[0]) # вычисление возраста клиента
    lst_time[i] = int(lst_time[i].split('T')[1][:2]) # вычисление времени операции в часах
    if (30 < lst_age[i] < 50) and not (7 < lst_time[i] < 23): # проверки условий
      res += [lst_trans[i]]
    elif (50 < lst_age[i] < 60) and not (7 < lst_time[i] < 22):
      res += [lst_trans[i]]
    elif (lst_age[i] >= 60) and not (8 < lst_time[i] < 22):
      res += [lst_trans[i]] # номер транзакции
  return list(set(res))

def short_reject(df): # паттерн 8: проверка на отказы, идущие подряд (ровно 3)
  lst_trans = list(df.index)
  lst_oper_res = list(df['oper_result'])
  res = []
  count = 0
  n = len(lst_trans)
  for i in range(n):
    if lst_oper_res[i] == 'Отказ':
      count += 1
    elif count < 3: # если успех и операций < 3, то сбрасываем счетчик
      count = 0
    if count == 3:
      res += lst_trans[i - count : i]
      count = 0
  return res

def reject(df): # паттерн 9: проверка на отказы, идущие подряд (больше 3х)
  lst_trans = list(df.index)
  lst_oper_res = list(df['oper_result'])
  res = []
  count = 0
  n = len(lst_trans)
  for i in range(n):
    if lst_oper_res[i] == 'Отказ':
      count += 1
    else:
      if count < 4:
        count = 0
      else:
        res += lst_trans[i - count : i]
    if i == n - 1 and count > 3: # если мы в конце списка операций, проверяем, выполнено ли условие
      res += lst_trans[i - count : i]
  return res


def result_frods(patts): # паттерн 11: итоговые фроды, с учетом паттернов и их весов
  score = [3, 1, 2, 4, 3, 2, 1, 2, 3, 2] # список весов каждого паттерна
  res_patts = []
  score_patt = {}
  all_frods = list(set(sum_list(patts))) # все операции, рассматриваемые, как фроды
  for trans in all_frods:
    score_patt[trans] = 0 # инициализация словаря
  for patt in all_frods:
    for i in range(len(patts)):
      if patt in patts[i]:
        score_patt[patt] += score[i] # добавление веса к транзакции в зависимости от принадлежности к паттерну
    if score_patt[patt] > 7:
      res_patts += [patt] # добавление в итоговый список
  return list(set(res_patts)), score_patt

# Вспомогательные функции

In [ ]:
def sum_list(A): # конкатенация списков, содержащихся в более крупном
  B = []
  for i in range(len(A)):
    B += A[i]
  return B

# Обработка данных и выделение паттернов

In [ ]:
num_patts = 9 # общее число базовых паттернов
patts = [[] for i in range(num_patts)]
func = [diff_city, not_valid_account, not_valid_passport, fast_trans, period, average, time_to_sleep, short_reject, reject] # функции фильтрации
for client in lst_clients:
  df = df_T[df_T['client'] == client]
  for i in range(num_patts):
    p = func[i](df)
    if p != None:
      patts[i] += p

num_patts += 1
patt10 = [] # паттерн 10 - совершение большого числа операций за малый промежуток времени
lst_addresses = list(set(list(df_T['address'])))
for addr in lst_addresses:
  df_addr = df_T[df_T['address'] == addr]
  patterns = fast_trans(df_addr)
  if patterns != None:
    patt10 += patterns
patts.append(patt10)

# добавление итогового паттерна
result_patts, patts_score = result_frods(patts)
patts.append(result_patts)
num_patts += 1

# вывод промежуточных результатов программы
for i in range(num_patts):
  print(f'patt {i+1} - number of transactions: {len(patts[i])}')
print(f'all frods: {len(sum_list(patts))}, unique: {len(set(sum_list(patts)))}')

patt 1 - number of transactions: 381
patt 2 - number of transactions: 662
patt 3 - number of transactions: 645
patt 4 - number of transactions: 7978
patt 5 - number of transactions: 7255
patt 6 - number of transactions: 3732
patt 7 - number of transactions: 3530
patt 8 - number of transactions: 2250
patt 9 - number of transactions: 4826
patt 10 - number of transactions: 7683
patt 11 - number of transactions: 7333
all frods: 46275, unique: 9953


# Загрузка данных в формате csv в .txt файл

In [ ]:
#подготовка данных к загрузке
tab = []
for i in range(num_patts):
  tab.append([i+1, sorted(patts[i], key=lambda x: x)])

In [ ]:
# создание DataFrame на основе наших данных
table = pd.DataFrame(tab, columns = ['№ Паттерна', 'Номера транзакций, попадающих под паттерн'])

In [ ]:
# проверка таблицы
table

,№ Паттерна,"Номера транзакций, попадающих под паттерн"
0,1,"[3649235841, 3649235870, 3649235871, 364923596..."
1,2,"[3649235865, 3649235866, 3649235867, 364923591..."
2,3,"[3649235842, 3649235868, 3649235959, 364923596..."
3,4,"[3649235840, 3649235841, 3649235844, 364923584..."
4,5,"[3649235844, 3649235845, 3649235846, 364923584..."
5,6,"[3649235844, 3649235845, 3649235847, 364923585..."
6,7,"[3649235842, 3649235843, 3649235844, 364923584..."
7,8,"[3649235845, 3649235846, 3649235847, 364923584..."
8,9,"[3649235846, 3649235847, 3649235847, 364923584..."
9,10,"[3649235844, 3649235845, 3649235846, 364923584..."


In [ ]:
# загрузка csv в .txt
with open('ответ.txt', "w+") as f:
  table.to_csv(f, index=False)

# Дашборды

In [ ]:
!pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# импротирование библиотек для визуализации
import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objs import *

Диаграмма 1

Количетсво транзакций с недействительным паспортом

In [ ]:
values = len(patts[2]) # кол-во клиентов с недействительным паспортом
all_clients = len(set(df_T['client'])) # всего клиентов

val = [values, all_clients - values]
names = ['Просрочен', 'Действителен']


fig = px.pie(values=val, names=names,color=names,title='Invalid passport',
             color_discrete_map={'Просрочен':'rgb(173, 255, 47)',
                                 'Действителен':'rgb(50, 205, 50)'
                                 })
fig.update_layout({
    'font': dict(
        size=23)
})

fig.show()

Диаграмма 2

Количетсво транзакций с недействительным аккаунтом

In [ ]:
values = len(patts[1]) # кол-во просроченных аккаунтов
all_accounts = len(set(df_T['account'])) #  всего аккаунтов

val = [values, all_clients - values]
names = ['Просрочен', 'Действителен']

fig = px.pie(values=val, names=names, color=names,title='Invalid account',
             color_discrete_map={'Просрочен':'rgb(0,134,149)',
                                 'Действителен':'rgb(0, 206, 209)'
                                 })
fig.update_layout({
    'font': dict(
        size=23)
})
fig.show()

Диаграмма 3

Успешные/неуспешные транзакции

In [ ]:
all_trans = len(df_T)
reject = (len(df_T[df_T['oper_result'] == 'Отказ']) /all_trans) * 100 

val = [reject, 100 - reject]
names = ['Просрочен', 'Действителен']

fig = px.pie(values=val, names=names, color=names, title='Before our selection',
             color_discrete_map={'Просрочен':'rgb(154, 205, 50)',
                                 'Действителен':'rgb(15,133,84)'
                                 })
fig.update_layout({
    'font': dict(
        size=23)
})
fig.show()

Диаграмма 4

Частота транзакций в определенное время

In [ ]:
# вычисление данных, необходимых для визуализации
lst_trans = list(df_T.index)
lst_sec = []
lst_date = list(df_T['date'])
res = [[], [], [], []]
for i in range(len(lst_date)): #  ниже время операции в секундах
  lst_sec.append(int(lst_date[i].split('T')[1].split(':')[-1]) + 60 *int(lst_date[i].split('T')[1].split(':')[1]) + 3600 * int(lst_date[i].split('T')[1][:2]) + 84600 * (int(lst_date[i].split('T')[0][-1]) - 1))
for i in range(1, len(lst_trans)):
  diff = lst_sec[i] - lst_sec[i-1] #вычисление разности времени транзакций
  if diff < 64: # проверка условия
    res[0] += [lst_trans[i]]
  elif diff < 79:
    res[1] += [lst_trans[i]]
  elif diff < 94:
    res[2] += [lst_trans[i]]
  else:
    res[3] += [lst_trans[i]]
for i in range(len(res)):
  res[i] = list(set(res[i])) # итоговые транзакции
vals = [100 * (len(i)/len(df_T)) for i in res] # перевод в проценты

In [ ]:
time_sec=['<64','65-79','80-94','>95']
color = ['#RF2019','rgb(0, 255, 0)','rgb(124, 252, 0)','rgb(127, 255, 0)']


fig = px.bar(x=time_sec, y=vals, labels={"y": "% transactions","x":"time <sec>"},color=time_sec,title='Transactions at a certain time')
fig.update_layout({
    'font': dict(
        size=23,
    )
})
fig.update_traces(marker_color='rgb(240, 230, 140)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5)
fig.show()

Диаграмма 5

Возрасто-временной анализ транзакций

In [ ]:
years = ["<30", "30-40", "40-50", "50-60", "60-70", "70-80", ">80"] # распределение по годам
Counts = [[0] * len(years) for i in range(7)]

lst_age = list(df_T['date_of_birth'])
lst_time = list(df_T['date'])
len_age = len(lst_age)
for i in range(len_age):
    lst_age[i] = int(lst_time[i].split('-')[0]) - int(lst_age[i].split('-')[0]) # вычисление возраста клиента
    lst_time[i] = int(lst_time[i].split('T')[1][:2]) # вычисление времени операции в часах

def count(from_, to_, counts):
  for i in range(len_age):
    if from_ <= lst_time[i] < to_:
      if lst_age[i] <= 30:
        counts[0] += 1
      elif lst_age[i] <= 40:
        counts[1] += 1
      elif lst_age[i] <= 50:
        counts[2] += 1
      elif lst_age[i] <= 60:
        counts[3] += 1
      elif lst_age[i] <= 70:
        counts[4] += 1
      elif lst_age[i] <= 80:
        counts[5] += 1
      else:
        counts[6] += 1

for i in range(7):
  count(i, i+1, Counts[i])

In [ ]:
time_oclock=['0-1','1-2','2-3','3-4','4-5','5-6','6-7','7-8']
color=['rgb(0, 0, 255)','rgb(154, 205, 50)','rgb(0, 206, 209)','rgb(0, 95, 0)','rgb(127, 255, 212)','rgb(50, 205, 50)','rgb(0, 0, 255)','rgb(70, 130, 180)']

fig = px.bar_polar()

def add_fig(count, name_,i):
  fig.add_trace(go.Barpolar(theta=time_oclock,
      r = count, 
      name = name_,
      marker_color=color[i]
      ))
  
for i in range(7):
  add_fig(Counts[i], years[i],i)

fig.update_layout({
    'font': dict(
        size=23),
    'polar': dict(
        radialaxis = dict(showticklabels=False)
    )
})
fig.show()

Диаграмма 6

Весовая характеристика фродов

In [ ]:
fraud_types = ['not fraud (<3)', 'may be fraud (4-7)', 'fraud (>8)']
fraud_lens = [0, 0, 0] #0-3, 4-7, 8-
for patt in list(patts_score.keys()):
  if patts_score[patt] < 3:
    fraud_lens[0] += 1
  elif patts_score[patt] < 7:
    fraud_lens[1] += 1
  else:
    fraud_lens[2] += 1

fraud_lens[0] += len(df_T) - len(list(patts_score.keys()))

fig = px.pie(fraud_lens,values=fraud_lens, names=fraud_types, color=fraud_types,
             color_discrete_map={'fraud (>8)':'rgb(30, 144, 255)',
                                 'may be fraud (4-7)':'rgb(135, 206, 235)',
                                 'not fraud (<3)':'lightcyan'
                                 })
fig.update_layout({
    'font': dict(
        size=23)
})
fig.show()

Диаграмма 7

Процентное отношение всех паттернов и найденных фродов

In [ ]:
size_tab = len(df_T)
groups = [i + 1 for i in range(num_patts)]
counts = [100 * (len(patt)/size_tab) for patt in patts]

fig = px.bar(x=groups, y=counts ,labels={"y": "number of transactions, %","x":"pattern number"})
                                
fig.update_layout({
    'font': dict(
        size=23)
})
fig.update_traces(marker_color='rgb(0, 191, 255)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5) 
fig.show()